In [ ]:
%cd ..

In [ ]:
import json
import numpy as np
import pandas as pd
from notebooks.utils import *
pd.set_option('display.max_rows', 1000)

In [ ]:
TEX_TABLE_DIR = "../masters-thesis-tex/tables/"

MODEL_COL = "model"
METHOD_COL = "method"
DATASET_COL = "dataset"
IMAGE_COL = "image_index"
SCALE_COL = "scale_factor"
DOWNSCALE_COL = "downscale_method"
METRIC_COL = "metric"
VALUE_COL = "value"

METHOD_MAPPING = {
    "bicubic": "Bicubic",
    "edsr": "EDSR \cite{lim_enhanced_2017}",
    "esrgan": "ESRGAN \cite{wang_esrgan_2019}",
    "dmsp_orig": "DMSP \cite{bigdeli_deep_2017}",
    "ircnn": "IRCNN \cite{zhang_learning_2017}",
    "dmsp_paper": "DMSP \cite{bigdeli_deep_2017}",
    "dpir": "DPIR \cite{zhang_plug-and-play_2021}",
    "dmsp": "DMSP",
    "hqs": "HQS",
}
METHOD_MAPPING_BLUR = METHOD_MAPPING.copy()
METHOD_MAPPING_BLUR.update(
    {
        "edsr": "EDSR\\bic\\ \cite{lim_enhanced_2017}",
        "esrgan": "ESRGAN\\bic\\ \cite{wang_esrgan_2019}",
    }
)


In [ ]:
hqs_folder = "results/sisr/hqs/"
dmsp_folder = "results/sisr/dmsp/"
bicubic_csv = "results/sisr/bicubic.csv"
edsr_csv = "results/sisr/edsr.csv"
esrgan_csv = "results/sisr/esrgan.csv"
dmsppaper_csv = "results/sisr/dmsp_paper.csv"
dpir_csv = "results/sisr/dpir.csv"

# HQS
df_hqs = load_folder(hqs_folder, MODEL_COL)
df_hqs[METHOD_COL] = "hqs"

# DMSP
df_dmsp = load_folder(dmsp_folder, MODEL_COL)
df_dmsp[METHOD_COL] = "dmsp"

# Bicubic
df_bicubic = pd.read_csv(bicubic_csv)
df_bicubic[METHOD_COL] = "bicubic"
df_bicubic[MODEL_COL] = ""

# EDSR
df_edsr = pd.read_csv(edsr_csv)
df_edsr[METHOD_COL] = "edsr"
df_edsr[MODEL_COL] = ""

# ESRGAN
df_esrgan = pd.read_csv(esrgan_csv)
df_esrgan[METHOD_COL] = "esrgan"
df_esrgan[MODEL_COL] = ""

# DMSP Paper
df_dmsp_paper = pd.read_csv(dmsppaper_csv)
df_dmsp_paper[METHOD_COL] = "dmsp_paper"
df_dmsp_paper[MODEL_COL] = ""

# DPIR
df_dpir = pd.read_csv(dpir_csv)
df_dpir[METHOD_COL] = "dpir"
df_dpir[MODEL_COL] = ""

# Combine the datasets
df_all = pd.concat([df_hqs, df_dmsp, df_bicubic, df_edsr, df_esrgan, df_dmsp_paper, df_dpir], axis=0)


In [ ]:
# Read the model name mapping
with open('models/model_name_mapping.json') as f:
    model_mapping = json.load(f)

model_mapping = {
    "": "",
    **model_mapping
}
models = list(model_mapping.keys())

In [ ]:
print(df_all[DATASET_COL].unique())

# Main Text Table

## Bicubic

In [ ]:
dataset = "Set5"
downscale = "bicubic"
metrics = ["PSNR", "LPIPS_ALEX"]
metrics_mapping = {
    "PSNR": "PSNR",
    "LPIPS_ALEX": "LPIPS",
}
scales = [2, 3, 4, 5]
methods = ["bicubic", "edsr", "esrgan", "dmsp_orig", "ircnn", "dmsp", "hqs"]
models = [
    "",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "drunet+_0.05",
    "drunet+_0.0-0.2",
    # "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]


In [ ]:
df = df_all.copy()

# Filter only the configured dataset
df = df[df[DATASET_COL] == dataset]
df = df[df[DOWNSCALE_COL] == downscale]
df = df.drop(DATASET_COL, axis=1)
df = df.drop(DOWNSCALE_COL, axis=1)

# Mean over the images
df = df.groupby([METHOD_COL, MODEL_COL, SCALE_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[METHOD_COL, MODEL_COL, SCALE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[MODEL_COL].isin(models)]
df = df[df[SCALE_COL].isin(scales)]
df = df[df[METHOD_COL].isin(methods)]

# Sort by Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(SCALE_COL, kind="mergesort")

# Remap metrics
df[METRIC_COL] = df[METRIC_COL].map(metrics_mapping)

# Noise Level and Metric as columns
df = df.set_index([METHOD_COL, MODEL_COL, SCALE_COL, METRIC_COL])
df = df.unstack([SCALE_COL, METRIC_COL])

df = df.reset_index()
df_add = pd.DataFrame(
    [
        ["dmsp_orig", "", 35.16, np.nan, 31.38, np.nan, 29.16, np.nan, 27.38, np.nan],
        ["ircnn", "", 35.07, np.nan, 31.26, np.nan, 29.01, np.nan, 27.13, np.nan],
    ],
    columns=df.columns,
)
df = pd.concat([df, df_add], axis=0)

# Sort and remap models
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df = df.sort_values(METHOD_COL, key=sort_key_for(methods), kind="mergesort")
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)
df[METHOD_COL] = df[METHOD_COL].map(METHOD_MAPPING)
df = df.set_index([METHOD_COL, MODEL_COL])

# Organize the column naming
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["Scaling factor \( s \)", ""])

# Rename model column
df.index = df.index.rename(["Method", ""])

df


### Convert to LaTeX

In [ ]:
column_format = "ll" + "C{1}" * len(df.columns)

formatters = [
    mark_column_best_formatter(
        df, c, mark_max=c[1] == "PSNR", num_decimals=2 if c[1] == "PSNR" else 4
    )
    for c in df.columns
]

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        # float_format=float_format(True),
        formatters=formatters,
        escape=False,
        column_format=column_format,
        multicolumn_format="c",
        multirow=True,
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

# Use multicolumn sometimes
def to_multicolumn(latex, text, size):
    return latex.replace(f"{text} &", f"\\multicolumn{{{size}}}{{l}}{{{text}}}")

latex = to_multicolumn(latex, METHOD_MAPPING["bicubic"], 2)
latex = to_multicolumn(latex, METHOD_MAPPING["edsr"], 2)
latex = to_multicolumn(latex, METHOD_MAPPING["esrgan"], 2)
latex = to_multicolumn(latex, METHOD_MAPPING["dmsp_orig"], 2)
latex = to_multicolumn(latex, METHOD_MAPPING["ircnn"], 2)
latex = to_multicolumn(latex, "Method", 2)

latex = add_midrule(latex, 9)
latex = add_midrule(latex, 12)

# print(latex)


In [ ]:
with open(TEX_TABLE_DIR + "sisr-bicubic-set5.tex", "w") as f:
    f.write(latex)

## Blur Kernels

In [ ]:
dataset = "Set5"
downscale = {
    **{f"kernel_{i}": "isotropic" for i in [0, 1, 2, 3]},
    **{f"kernel_{i}": "anisotropic" for i in [4, 5, 6, 7]}
}
metrics = ["PSNR", "LPIPS_ALEX"]
metrics_mapping = {
    "PSNR": "PSNR",
    "LPIPS_ALEX": "LPIPS",
}
scales = [2, 3]
methods = ["bicubic", "edsr", "dpir", "dmsp", "hqs"]
models = [
    "",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "drunet+_0.05",
    "drunet+_0.0-0.2",
    # "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]


In [ ]:
df = df_all.copy()

# Filter only the configured dataset
df = df[df[DATASET_COL] == dataset]
df = df.drop(DATASET_COL, axis=1)

# Map the downscale column
df = df[df[DOWNSCALE_COL].isin(downscale)]
df[DOWNSCALE_COL] = df[DOWNSCALE_COL].map(downscale)

# Mean over the images
df = df.groupby([METHOD_COL, MODEL_COL, SCALE_COL, DOWNSCALE_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[METHOD_COL, MODEL_COL, SCALE_COL, DOWNSCALE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[MODEL_COL].isin(models)]
df = df[df[SCALE_COL].isin(scales)]
df = df[df[METHOD_COL].isin(methods)]

# Sort by Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(SCALE_COL, kind="mergesort")
df = df.sort_values(
    DOWNSCALE_COL, key=sort_key_for(list(downscale.values())), kind="mergesort"
)

# Remap metrics
df[METRIC_COL] = df[METRIC_COL].map(metrics_mapping)

# Noise Level and Metric as columns
df = df.set_index([METHOD_COL, MODEL_COL, DOWNSCALE_COL, SCALE_COL, METRIC_COL])
df = df.unstack([DOWNSCALE_COL, SCALE_COL, METRIC_COL])

# Sort and remap models
df = df.reset_index()
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df = df.sort_values(METHOD_COL, key=sort_key_for(methods), kind="mergesort")
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)
df[METHOD_COL] = df[METHOD_COL].map(METHOD_MAPPING_BLUR)
df = df.set_index([METHOD_COL, MODEL_COL])

# Organize the column naming
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["Blur kernels", "Scaling factor \( s \)", ""])

# Rename model column
df.index = df.index.rename(["Method", ""])

df


### Convert to LaTeX

In [ ]:
column_format = "ll" + "C{1}" * len(df.columns)

formatters = [
    mark_column_best_formatter(
        df, c, mark_max=c[2] == "PSNR", num_decimals=2 if c[2] == "PSNR" else 4
    )
    for c in df.columns
]

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        # float_format=float_format(True),
        formatters=formatters,
        escape=False,
        column_format=column_format,
        multicolumn_format="c",
        multirow=True,
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

# Use multicolumn sometimes
def to_multicolumn(latex, text, size):
    return latex.replace(f"{text} &", f"\\multicolumn{{{size}}}{{l}}{{{text}}}")

latex = to_multicolumn(latex, METHOD_MAPPING_BLUR["bicubic"], 2)
latex = to_multicolumn(latex, METHOD_MAPPING_BLUR["edsr"], 2)
latex = to_multicolumn(latex, METHOD_MAPPING_BLUR["esrgan"], 2)
latex = to_multicolumn(latex, "Method", 2)

latex = add_midrule(latex, 10)

# print(latex)


In [ ]:
with open(TEX_TABLE_DIR + "sisr-blur-set5.tex", "w") as f:
    f.write(latex)

# Appendix Table

## Bicubic

In [ ]:
datasets = {"Set5": "Set5", "Set14": "Set14", "CBSD68": "CBSD68 \cite{martin_database_2001}"}
downscale = "bicubic"
metrics = {
    "PSNR": "P",
    "SSIM": "S",
    "FSIM": "F",
    "LPIPS_ALEX": "L",
}
scales = [2, 3, 4, 5]
methods = ["bicubic", "edsr", "esrgan", "dmsp", "hqs"]
models = [
    "",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "drunet+_0.05",
    "drunet+_0.0-0.2",
    # "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]


In [ ]:
df = df_all.copy()

df = df[df[DOWNSCALE_COL] == downscale]
df = df.drop(DOWNSCALE_COL, axis=1)

# Mean over the images
df = df.groupby([METHOD_COL, MODEL_COL, SCALE_COL, DATASET_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[METHOD_COL, MODEL_COL, DATASET_COL, SCALE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[DATASET_COL].isin(datasets)]
df = df[df[METHOD_COL].isin(methods)]
df = df[df[MODEL_COL].isin(models)]

# Remap names
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df = df.sort_values(METHOD_COL, key=sort_key_for(methods), kind="mergesort")
df[METHOD_COL] = df[METHOD_COL].map(METHOD_MAPPING)
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)

# Models as columns
df = df.set_index([DATASET_COL, SCALE_COL, METRIC_COL, METHOD_COL, MODEL_COL])
df = df.unstack([METHOD_COL, MODEL_COL])

df = df.reset_index()

# Sort by Dataset, Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(SCALE_COL, kind="mergesort")
df = df.sort_values(DATASET_COL, key=sort_key_for(datasets), kind="mergesort")

# Rename Dataset, Noise, and Metric
df[DATASET_COL] = df[DATASET_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{" + datasets[x] + "}"
)
df[SCALE_COL] = df[SCALE_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{s=" + str(x) + " }"
)
df[METRIC_COL] = df[METRIC_COL].map(metrics)

df = df.set_index([DATASET_COL, SCALE_COL, METRIC_COL])

# Update the column index and index
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["", ""])
df.index = df.index.rename(["", "", ""])

# Replace nan
df = df.replace(np.nan, "---")

df


In [ ]:
column_format = "lll|" + "C{1}C{1}C{1}|" + "C{1}" * 6 + "|" + "C{1}" * 2

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        float_format=float_format(True),
        escape=False,
        na_rep="---",
        column_format=column_format,
        multicolumn_format="c",
        multirow=True
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

# Bug in pandas?? "&" Missing in line 4
lines = latex.splitlines()
lines[3] = "& & &" + lines[3]
latex = "\n".join(lines)

latex = delete_line(latex, 4)

# print(latex)

In [ ]:
with open(TEX_TABLE_DIR + "all_sisr_bicubic.tex", "w") as f:
    f.write(latex)

## Blur

In [ ]:
dataset = "Set5"
downscale = {f"kernel_{i}": f"({chr(97+i)})" for i in range(8)}
metrics = {
    "PSNR": "P",
    # "SSIM": "S",
    # "FSIM": "F",
    "LPIPS_ALEX": "L",
}
scales = [2, 3, 4]
methods = ["bicubic", "edsr", "esrgan", "dpir", "dmsp", "hqs"]
models = [
    "",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "drunet+_0.05",
    "drunet+_0.0-0.2",
    # "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]


In [ ]:
df = df_all.copy()

# Filter only the configured dataset
df = df[df[DATASET_COL] == dataset]
df = df.drop(DATASET_COL, axis=1)

# Map the downscale column
df = df[df[DOWNSCALE_COL].isin(downscale)]
# df[DOWNSCALE_COL] = df[DOWNSCALE_COL].map(downscale)

# Mean over the images
df = df.groupby([METHOD_COL, MODEL_COL, DOWNSCALE_COL, SCALE_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[METHOD_COL, MODEL_COL, DOWNSCALE_COL, SCALE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[SCALE_COL].isin(scales)]
df = df[df[METHOD_COL].isin(methods)]
df = df[df[MODEL_COL].isin(models)]

# Remap names
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df = df.sort_values(METHOD_COL, key=sort_key_for(methods), kind="mergesort")
df[METHOD_COL] = df[METHOD_COL].map(METHOD_MAPPING_BLUR)
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)

# Models as columns
df = df.set_index([DOWNSCALE_COL, SCALE_COL, METRIC_COL, METHOD_COL, MODEL_COL])
df = df.unstack([METHOD_COL, MODEL_COL])

df = df.reset_index()

# Sort by Dataset, Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(SCALE_COL, kind="mergesort")
df = df.sort_values(DOWNSCALE_COL, kind="mergesort")

# Rename Dataset, Noise, and Metric
df[DOWNSCALE_COL] = df[DOWNSCALE_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{" + downscale[x] + "}"
)
df[SCALE_COL] = df[SCALE_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{s=" + str(x) + " }"
)
df[METRIC_COL] = df[METRIC_COL].map(metrics)

df = df.set_index([DOWNSCALE_COL, SCALE_COL, METRIC_COL])

# Update the column index and index
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["", ""])
df.index = df.index.rename(["", "", ""])

# Replace nan
df = df.replace(np.nan, "---")

df


In [ ]:
# column_format = "lll|" + "C{1}C{1}C{1}C{1}|" + "C{1}" * 6 + "|" + "C{1}" * 2
column_format = "lll|" + "C{0.8}" * 4 + "|" + "C{0.9}" * 4 + "C{1}" * 2 + "|" + "C{1}" * 2

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        float_format=float_format(True),
        escape=False,
        na_rep="---",
        column_format=column_format,
        multicolumn_format="c",
        multirow=True
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

# Bug in pandas?? "&" Missing in line 4
lines = latex.splitlines()
lines[3] = "& & & & " + lines[3]
latex = "\n".join(lines)

latex = delete_line(latex, 4)

# print(latex)

In [ ]:
with open(TEX_TABLE_DIR + "all_sisr_blur.tex", "w") as f:
    f.write(latex)